In [1]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from fuzzywuzzy import fuzz

from dotenv import load_dotenv

load_dotenv()

# Get credentials from environment variables
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

In [2]:
def find_song_album(song_name, artist_name):
    """
    Finds the best matching song and album for a given song name and artist name using Spotify's API.

    Parameters:
    - song_name (str): The name of the song to search for.
    - artist_name (str): The name of the artist to search for.
    - client_id (str): Spotify API client ID.
    - client_secret (str): Spotify API client secret.

    Returns:
    - dict: A dictionary containing the best match's song name, artist name, album name, release date, and match score.
    - None: If no match is found with a score above the threshold.
    """
    
    # Initialize Spotify client with provided credentials
    client_credentials_manager = SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    )
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    
    # Construct the search query for Spotify API
    query = f"track:{song_name} artist:{artist_name}"
    # Perform the search with a limit of 50 results
    results = sp.search(q=query, type='track', limit=50)
    
    best_match = None
    highest_score = 0
    
    # Iterate over each track in the search results
    for track in results['tracks']['items']:
        # Calculate the similarity score for the song name
        song_score = fuzz.ratio(song_name.lower(), track['name'].lower())
        # Calculate the similarity score for the artist name
        artist_score = fuzz.ratio(artist_name.lower(), track['artists'][0]['name'].lower())
        
        # Calculate a combined score with weighted contributions from song and artist scores
        combined_score = (song_score * 0.6) + (artist_score * 0.4)
        
        # Update the best match if the current track has a higher combined score
        if combined_score > highest_score:
            highest_score = combined_score
            best_match = track
    
    # Check if a suitable match was found with a score above the threshold
    if best_match and highest_score > 70:  # Threshold for considering a match
        return {
            'song': best_match['name'],
            'artist': best_match['artists'][0]['name'],
            'album': best_match['album']['name'],
            'release_date': best_match['album']['release_date'],
            'match_score': highest_score
        }
    else:
        # Return None if no match meets the threshold criteria
        return None

In [3]:
result = find_song_album("Seed", "AURORA")
result


{'song': 'The Seed',
 'artist': 'AURORA',
 'album': 'A Different Kind Of Human (Step II)',
 'release_date': '2019-06-07',
 'match_score': 80.19999999999999}